In [2]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0,openai_api_key=os.getenv("OPENAI_API_KEY"))

# Method 1

In [25]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str 
    film_names: List[str] 


actor_query = """
        Generate the filmography for a random actor.

        Output Format:
        {{
            name: <name of actor>
            film_names: [<film_name>, <film_name>, <film_name>]
        }}
    """

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{query}\n",
    input_variables=["query"],
)

chain = prompt | llm | parser

chain.invoke({"query": actor_query})

Actor(name='Leonardo DiCaprio', film_names=['Titanic', 'Inception', 'The Revenant', 'The Wolf of Wall Street', 'Shutter Island'])

# Method 2

In [ ]:

load_dotenv()
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0,openai_api_key=os.getenv("OPENAI_API_KEY"))

# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": actor_query})

Actor(name='Leonardo DiCaprio', film_names=['Titanic', 'Inception', 'The Revenant', 'The Wolf of Wall Street', 'Shutter Island', 'Catch Me If You Can', 'Once Upon a Time in Hollywood', 'The Aviator', 'Django Unchained', 'Gangs of New York'])

# Method 3

In [29]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

chain = prompt | llm.with_structured_output(Actor) 

chain.invoke({"query": actor_query})

Actor(name='Leonardo DiCaprio', film_names=["What's Eating Gilbert Grape", 'The Aviator', 'The Departed', 'Inception', 'The Wolf of Wall Street', 'Once Upon a Time in Hollywood'])